In [27]:
import gpxpy
import json

import pandas as pd
import numpy as np

from geopy.distance import geodesic

In [28]:
def get_gpx_df(gpx_file):
    with open(gpx_file, 'r') as file:
        gpx = gpxpy.parse(file)

    data = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                data.append({
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                })

    df = pd.DataFrame(data)

    df['prev_latitude'] = df['latitude'].shift(1)
    df['prev_longitude'] = df['longitude'].shift(1)
    
    df['distance'] = df.apply(lambda x: geodesic((x['prev_latitude'], x['prev_longitude']), (x['latitude'], x['longitude'])).meters if not pd.isna(x['prev_latitude']) and not pd.isna(x['prev_longitude']) else 0, axis=1)
    df['accumulated_distance'] = df['distance'].cumsum()

    df = df.drop(columns=[
        'prev_latitude', 
        'prev_longitude',
        'distance'
    ])
    
    df.rename(columns={'accumulated_distance': 'distance'}, inplace=True)
    
    return df

In [29]:
df = get_gpx_df('../public/track.gpx')
df.head(3)

,latitude,longitude,distance
0,1.34228,103.83519,0.000000
1,1.34265,103.83574,73.616069
2,1.34276,103.83592,97.054093


In [30]:
segments, current_segment = [], []
distance, total_km = 0, 0
for i, row in df.iterrows():
    current_segment.append((row['latitude'], row['longitude']))
    current_km = row['distance'] // 1000.0
    
    if current_km > total_km:
        distance = round(row['distance'] / 1000.0, 2)
        segments.append((current_segment, distance))
        
        current_segment = []
        total_km = current_km

if current_segment:
    segments.append((current_segment, distance))

def calculate_midpoint(segment):
    latitudes = [point[0] for point in segment]
    longitudes = [point[1] for point in segment]
    return np.mean(latitudes), np.mean(longitudes)

# Calculate midpoints for each segment
midpoints = [(calculate_midpoint(segment[0]), segment[1]) for segment in segments]

for (midpoint, distance) in midpoints:
    print(f"{distance}km: {midpoint[0]}, {midpoint[1]}")

data = [{"name": f"{distance}km", "lat": midpoint[0], "lon": midpoint[1]} for (midpoint, distance) in midpoints]    
    
with open('output.json', 'w') as f:
    json.dump(data, f, indent=4)

1.09km: 1.3448884174591158, 103.83750937333922
2.02km: 1.352199103417141, 103.83464767571006
3.02km: 1.357059046118097, 103.82764541290022
4.04km: 1.3586753785078014, 103.8253666936819
5.01km: 1.3643204056742517, 103.82782722399993
6.0km: 1.3727048140551363, 103.82702996022999
7.03km: 1.3787256740033627, 103.82563367885139
8.01km: 1.3808077728026547, 103.81937808415387
9.02km: 1.387241576622064, 103.81782050496219
10.15km: 1.3938690835367078, 103.81828212867613
11.1km: 1.4008483961224556, 103.81941395252943
12.04km: 1.4032260248703616, 103.82741389031123
13.01km: 1.4094112981110811, 103.83140706457198
14.13km: 1.4110607956536114, 103.83474218659103
15.0km: 1.4156649667503578, 103.84462495800108
16.0km: 1.4218809669837356, 103.85022095963359
17.04km: 1.4256808745928786, 103.8550681349906
18.04km: 1.42018937615349, 103.86272229802083
19.0km: 1.4130118332104757, 103.86151058482938
20.03km: 1.405989478475281, 103.86087519914976
21.05km: 1.4064819663763046, 103.86759396269917
22.03km: 1.412